# 🏥 Medical Image Segmentation - TransUNet Training

This notebook trains **TransUNet** models on different data fractions (10%, 25%, 50%, 100%).

**Runtime:** ~3-4 hours on Colab GPU (TransUNet is slower due to transformer)

---

## Step 1: Setup GPU Runtime

⚠️ **IMPORTANT:** Make sure you're using a GPU runtime!
- Go to: **Runtime → Change runtime type → GPU**

In [ ]:
# Verify GPU is available
import torch
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ WARNING: No GPU detected! Training will be very slow.")

## Step 2: Clone Repository

In [ ]:
# Clone the repository
!git clone https://github.com/Prabhat9801/Medical-Image-Segmentation.git
%cd Medical-Image-Segmentation

## Step 3: Install Dependencies

In [ ]:
# Install required packages
!pip install -q timm albumentations opencv-python-headless tqdm

## Step 4: Mount Google Drive & Extract Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile
import os
import shutil
import pandas as pd

# Paths
zip_path = "/content/drive/MyDrive/isic_processed_256.zip"
final_path = "/content/Medical-Image-Segmentation/data/processed/isic"

print("📦 Extracting data with Windows path fix...")

# Remove existing if present
if os.path.exists(final_path):
    shutil.rmtree(final_path)

# Create the directory structure
os.makedirs(final_path, exist_ok=True)
os.makedirs(os.path.join(final_path, "images"), exist_ok=True)
os.makedirs(os.path.join(final_path, "masks"), exist_ok=True)

# Extract and fix Windows paths
with zipfile.ZipFile(zip_path, 'r') as z:
    for file_info in z.filelist:
        orig_name = file_info.filename
        fixed_name = orig_name.replace('\\', '/')
        
        if fixed_name.startswith('isic/'):
            fixed_name = fixed_name[5:]
        
        if 'images/' in fixed_name or (fixed_name.endswith('.png') and 'mask' not in fixed_name.lower()):
            filename = os.path.basename(fixed_name)
            target_path = os.path.join(final_path, "images", filename)
        elif 'masks/' in fixed_name or '_mask.png' in fixed_name:
            filename = os.path.basename(fixed_name)
            target_path = os.path.join(final_path, "masks", filename)
        elif fixed_name.endswith('splits.csv'):
            target_path = os.path.join(final_path, "splits.csv")
        else:
            continue
        
        with z.open(file_info) as source, open(target_path, 'wb') as target:
            shutil.copyfileobj(source, target)

print("✅ Extraction complete!")

# Fix splits.csv paths
print("\n🔧 Fixing splits.csv paths...")
splits_path = os.path.join(final_path, "splits.csv")
if os.path.exists(splits_path):
    df = pd.read_csv(splits_path)
    
    # Fix paths to absolute Colab paths
    df['image_path'] = df['image_path'].apply(
        lambda x: os.path.join(final_path, "images", os.path.basename(x))
    )
    df['mask_path'] = df['mask_path'].apply(
        lambda x: os.path.join(final_path, "masks", os.path.basename(x))
    )
    
    df.to_csv(splits_path, index=False)
    print("✅ splits.csv paths fixed!")

# Verify
import glob
image_count = len(glob.glob(os.path.join(final_path, "images", "*.png")))
mask_count = len(glob.glob(os.path.join(final_path, "masks", "*.png")))

print(f"\n✅ Images: {image_count}")
print(f"✅ Masks: {mask_count}")
print(f"✅ splits.csv: {'Found' if os.path.exists(splits_path) else 'NOT FOUND'}")
print("\n🎉 Data extraction successful!")

## Step 5: Train TransUNet - 10% Data

**Expected time:** ~20-30 minutes

In [ ]:
!python -m src.train \
    --model transunet \
    --epochs 50 \
    --batch_size 8 \
    --data_fraction 0.1 \
    --lr 1e-4

print("\n✅ TransUNet 10% training complete!")

## Step 6: Train TransUNet - 25% Data

**Expected time:** ~40-50 minutes

In [ ]:
!python -m src.train \
    --model transunet \
    --epochs 50 \
    --batch_size 8 \
    --data_fraction 0.25 \
    --lr 1e-4

print("\n✅ TransUNet 25% training complete!")

## Step 7: Train TransUNet - 50% Data

**Expected time:** ~60-80 minutes

In [ ]:
!python -m src.train \
    --model transunet \
    --epochs 50 \
    --batch_size 8 \
    --data_fraction 0.5 \
    --lr 1e-4

print("\n✅ TransUNet 50% training complete!")

## Step 8: Train TransUNet - 100% Data

**Expected time:** ~120-150 minutes

In [ ]:
!python -m src.train \
    --model transunet \
    --epochs 50 \
    --batch_size 8 \
    --data_fraction 1.0 \
    --lr 1e-4

print("\n✅ TransUNet 100% training complete!")

## Step 9: Save Results to Google Drive

In [ ]:
# Copy all experiments to Google Drive
!mkdir -p /content/drive/MyDrive/medical_segmentation_results
!cp -r experiments /content/drive/MyDrive/medical_segmentation_results/transunet_experiments

print("\n✅ All TransUNet results saved to Google Drive!")
print("📁 Location: /content/drive/MyDrive/medical_segmentation_results/transunet_experiments")

## 🎉 TransUNet Training Complete!

**Next step:**
- Run `COLAB_RESULTS.ipynb` to evaluate and compare all models